<a href="https://colab.research.google.com/github/ali-kmirzaei/YOLO/blob/main/train.YOLOv5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Initialization:**

In [ ]:
!git clone https://github.com/ultralytics/yolov5 

In [ ]:
%cd yolov5
!pip install -r requirements.txt 

In [3]:
# import dataset from google drive
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive


Mounted at /content/gdrive
 3000.weights	  code.zip	     last.pt	   Yolov3-2.mp4
 4001419029.zip  'Colab Notebooks'   newPack.zip   YOLOv3.mp4
 5900.weights	  empty.zip	     obj0.zip	   yolov3_training_6000.weights
 800.weights	  extra.zip	     train.zip	   YOLOv5Performance


In [36]:
!cp -rf /mydrive/YOLOv5Performance/runs/* runs/

In [ ]:
!unzip /mydrive/newPack.zip -d data

In [ ]:
!cp /mydrive/last.pt data/last_trained.pt

In [ ]:
!unzip /mydrive/extra.zip -d data/images/extra_test

# **split train and test files:**

In [5]:
import os
import glob
import random
import shutil

In [6]:
!rm -rf data/images
!rm -rf data/labels

!mkdir data/images
!mkdir data/images/test
!mkdir data/images/train

!mkdir data/labels
!mkdir data/labels/test
!mkdir data/labels/train

In [7]:
image_list = list()
label_list = list()
total_tuples = list()

total = glob.glob('data/newPack/*')
total.sort()

for i in range(0, len(total)-1, 2):
  total_tuples.append((total[i], total[i+1]))

# # validate list:
# for tpl in total_tuples:
#   if tpl[0][13: 23] != tpl[1][13: 23]:
#     print("ERROR")

random.shuffle(total_tuples)

num = 1085
for i in range(num):
  # IMG
  src = total_tuples[i][0]
  dst = 'data/images/train/'+src[13:]
  shutil.copyfile(src, dst)
  # LBL
  src = total_tuples[i][1]
  dst = 'data/labels/train/'+src[13:]
  shutil.copyfile(src, dst)

for i in range(num, 1550):
  # IMG
  src = total_tuples[i][0]
  dst = 'data/images/test/'+src[13:]
  shutil.copyfile(src, dst)
  # LBL
  src = total_tuples[i][1]
  dst = 'data/labels/test/'+src[13:]
  shutil.copyfile(src, dst)

In [31]:
# Check Distributions

alls = glob.glob('data/newPack/*.jpg')
trains = glob.glob('data/images/train/*')
tests = glob.glob('data/images/test/*')
nums = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0']

cnt = 0
for all in alls:
  if all[13] in nums:
    cnt += 1
all_per = 100 * cnt / len(alls)


cnt = 0
for test in tests:
  if test[18] in nums:
    cnt += 1
test_per = 100 * cnt / len(tests)

cnt = 0
for train in trains:
  if train[19] in nums:
    cnt += 1
train_per = 100 * cnt / len(trains)

print(all_per, test_per, train_per)

58.064516129032256 57.204301075268816 58.433179723502306


# **train and test and visualization model :**

In [40]:
!rm -rf runs/train/yolo_idcard32

In [66]:
# TRAIN
!python train.py --img 640 --cfg data/yolov5s.yaml --batch 64 --epochs 100 --data idcard_data.yaml --weights yolov5s.pt --name yolo_idcard3 --cache

train: weights=yolov5s.pt, cfg=data/yolov5s.yaml, data=idcard_data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=64, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=yolo_idcard3, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-132-g014acde torch 1.10.0+cu111 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degree

In [ ]:
# Free Space!

In [67]:
# TEST
!python val.py --data idcard_data.yaml --weights runs/train/yolo_idcard35/weights/last.pt --name yolo_idcard3 --task val

val: data=/content/yolov5/data/idcard_data.yaml, weights=['runs/train/yolo_idcard35/weights/last.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=yolo_idcard3, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.1-132-g014acde torch 1.10.0+cu111 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
YOLOv5s summary: 224 layers, 7067395 parameters, 0 gradients, 16.4 GFLOPs
val: Scanning '/content/yolov5/data/labels/test.cache' images and labels... 465 found, 0 missing, 0 empty, 0 corrupt: 100% 465/465 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 15/15 [00:13<00:00,  1.12it/s]
                 all        465       2791      0.978      0.982      0.981      0.743
                card        465        464      0.983      0.989      0.989      0.981
   

In [46]:
# Visualize
from utils.plots import plot_results
plot_results('results.csv')

In [ ]:
!python detect.py --source data/images/extra_test/ --weights runs/train/yolo_idcard35/weights/last.pt --conf 0.25 --name yolo_idcard_extra

In [78]:
!cp -r runs/detect/ /mydrive/YOLOv5Performance/new_detect

In [59]:
!mkdir /mydrive/YOLOv5Performance/200e/val

In [76]:
!rm -rf runs/detect/yolo_idcard_extra2